In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from tqdm import tqdm
import dateutil

m = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [2]:
data = []

for i in range(11, 12):
    for j in range(m[i]):
        
        print('2013-' + str(i+1) + '-' + str(j+1))
        browser = webdriver.Chrome()
        browser.get('https://www.wunderground.com/history/daily/gb/london/EGLC/date/2013-' + str(i+1) + '-' + str(j+1))
        
        sleep(10)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        table = soup.find_all('table')[2].find_all('tr')
        
        n = len(table)
        for k in range(1, n):
            tmp = ['2013-%02d-%02d' %(i+1, j+1), table[k].find_all('span')[0].get_text(), table[k].find_all(class_='wu-value wu-value-to')[0].get_text()]
            data.append(tmp)
        
        browser.quit()

2013-12-1
2013-12-2
2013-12-3
2013-12-4
2013-12-5
2013-12-6
2013-12-7
2013-12-8
2013-12-9
2013-12-10
2013-12-11
2013-12-12
2013-12-13
2013-12-14
2013-12-15
2013-12-16
2013-12-17
2013-12-18
2013-12-19
2013-12-20
2013-12-21
2013-12-22
2013-12-23
2013-12-24
2013-12-25
2013-12-26
2013-12-27
2013-12-28
2013-12-29
2013-12-30
2013-12-31


In [36]:
data = pd.DataFrame(data)
data.columns = ['Date', 'Time', 'T']
data['Date'] = data['Date'].astype('datetime64[ns]')
data.to_csv('../data/weather/weather_London_12.csv', index=None)

## Clean weather data (Cork)

In [29]:
data = pd.read_csv('../data/weather/weather_Cork.csv')
data['Date'] = data['Date'].astype('datetime64[ns]')

In [5]:
for i in tqdm(range(12)):
    for j in range(m[i]):
        tmp = data[data['Date'] == '2010-%02d-%02d' % (i+1, j+1)]['Time']
        
        if 'P' in tmp.values[0]:
            print('2010-%02d-%02d' % (i+1, j+1), tmp.values[0])
            k = 0
            while 'P' in tmp.values[k]:
                data['Date'][tmp.index[k]] = data['Date'][tmp.index[k]] - dateutil.relativedelta.relativedelta(days=1)
                k = k+1

  0%|          | 0/12 [00:00<?, ?it/s]/Users/kaishuai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
 25%|██▌       | 3/12 [00:00<00:00, 15.54it/s]

2010-03-29 11:00 PM
2010-03-30 11:00 PM
2010-03-31 11:00 PM
2010-04-01 11:00 PM
2010-04-02 11:00 PM
2010-04-03 11:00 PM
2010-04-04 11:00 PM
2010-04-05 11:00 PM
2010-04-06 11:00 PM
2010-04-07 11:00 PM
2010-04-08 11:00 PM
2010-04-09 11:00 PM
2010-04-10 11:00 PM
2010-04-11 11:00 PM
2010-04-12 11:00 PM
2010-04-13 11:00 PM
2010-04-14 11:00 PM
2010-04-15 11:00 PM
2010-04-16 11:00 PM
2010-04-17 11:00 PM
2010-04-18 11:00 PM
2010-04-19 11:00 PM
2010-04-20 11:00 PM
2010-04-21 11:00 PM
2010-04-22 11:00 PM
2010-04-23 11:00 PM
2010-04-24 11:00 PM
2010-04-25 11:00 PM


 33%|███▎      | 4/12 [00:01<00:03,  2.30it/s]

2010-04-26 11:00 PM
2010-04-27 11:00 PM
2010-04-28 11:00 PM
2010-04-29 11:00 PM
2010-04-30 11:00 PM
2010-05-01 11:00 PM
2010-05-02 11:00 PM
2010-05-03 11:00 PM
2010-05-04 11:00 PM
2010-05-05 11:00 PM
2010-05-06 11:00 PM
2010-05-07 11:00 PM
2010-05-08 11:00 PM
2010-05-09 11:00 PM
2010-05-10 11:00 PM
2010-05-11 11:00 PM
2010-05-12 11:00 PM
2010-05-13 11:00 PM
2010-05-14 11:00 PM
2010-05-15 11:00 PM
2010-05-16 11:00 PM
2010-05-17 11:00 PM
2010-05-18 11:00 PM
2010-05-19 11:00 PM
2010-05-20 11:00 PM
2010-05-21 11:00 PM
2010-05-22 11:00 PM
2010-05-23 11:00 PM
2010-05-24 11:00 PM
2010-05-25 11:00 PM
2010-05-26 11:00 PM
2010-05-27 11:00 PM
2010-05-28 11:00 PM


 42%|████▏     | 5/12 [00:02<00:05,  1.35it/s]

...

2010-08-30 11:00 PM
2010-08-31 11:00 PM
2010-09-01 11:00 PM
2010-09-02 11:00 PM
2010-09-03 11:00 PM
2010-09-04 11:00 PM
2010-09-05 11:00 PM
2010-09-06 11:00 PM
2010-09-07 11:00 PM
2010-09-08 11:00 PM
2010-09-09 11:00 PM
2010-09-10 11:00 PM
2010-09-11 11:00 PM
2010-09-12 11:00 PM
2010-09-13 11:00 PM
2010-09-14 11:00 PM
2010-09-15 11:00 PM
2010-09-16 11:00 PM
2010-09-17 11:00 PM
2010-09-18 11:00 PM
2010-09-19 11:00 PM
2010-09-20 11:00 PM
2010-09-21 11:00 PM
2010-09-22 11:00 PM
2010-09-23 11:00 PM
2010-09-24 11:00 PM
2010-09-25 11:00 PM
2010-09-26 11:00 PM
2010-09-27 11:00 PM


 75%|███████▌  | 9/12 [00:08<00:03,  1.18s/it]

2010-09-28 11:00 PM
2010-09-29 11:00 PM
2010-09-30 11:00 PM
2010-10-01 11:00 PM
2010-10-02 11:00 PM
2010-10-03 11:00 PM
2010-10-04 11:00 PM
2010-10-05 11:00 PM
2010-10-06 11:00 PM
2010-10-07 11:00 PM
2010-10-08 11:00 PM
2010-10-09 11:00 PM
2010-10-10 11:00 PM
2010-10-11 11:00 PM
2010-10-12 11:00 PM
2010-10-13 11:00 PM
2010-10-14 11:00 PM
2010-10-15 11:00 PM
2010-10-16 11:00 PM
2010-10-17 11:00 PM
2010-10-18 11:00 PM
2010-10-19 11:00 PM
2010-10-20 11:00 PM
2010-10-21 11:00 PM
2010-10-22 11:00 PM
2010-10-23 11:00 PM
2010-10-24 11:00 PM
2010-10-25 11:00 PM


100%|██████████| 12/12 [00:09<00:00,  1.26it/s]

2010-10-26 11:00 PM
2010-10-27 11:00 PM
2010-10-28 11:00 PM
2010-10-29 11:00 PM
2010-10-30 11:00 PM
2010-10-31 11:00 PM


In [261]:
# Datetime
datetime = []
for i in range(len(data)):
    datetime.append(pd.to_datetime(str(data['Date'][i]) + ' ' + data['Time'][i]))
    
data = pd.concat([data, pd.DataFrame(datetime, columns=['DateTime'])], axis=1)

In [262]:
# Transfer to DateTime
data = data[['DateTime', 'T']]
data.to_csv('../data/weather/weather_Cork_cleaned.csv', index=None)
data = pd.read_csv('../data/weather/weather_Cork_cleaned.csv')

In [263]:
# Delete rows with T == 0
for i in range(len(data)):
    if data['T'][i] == 0:
        data.drop(index=[i], inplace=True)

In [264]:
data['Miss'] = 0

In [265]:
# Construct the empty DataFrame to store the interpolation
add = pd.DataFrame(columns=['DateTime', 'T', 'Miss'])

for j in range(1, 365*48):
    t = pd.to_datetime('2010-01-01 00:00:00') + dateutil.relativedelta.relativedelta(minutes=30*j)
    if str(t) not in data['DateTime'].values:

        # Use average values as interpolation
        value = (data[data['DateTime'] > str(t)].iloc[0, 1] + data[data['DateTime'] < str(t)].iloc[-1, 1])/2
        tmp = pd.DataFrame([[str(t), value, 1]], columns=['DateTime', 'T', 'Miss'])
        add = pd.concat([add, tmp], axis=0)

In [266]:
data = pd.concat([data, add]).sort_values(by='DateTime')
data.reset_index(inplace=True, drop=True)
data.to_csv('../data/weather/weather_Cork_cleaned.csv', index=None)

## Clean weather data (Dublin)

In [46]:
data = pd.read_csv('../data/weather/weather_Dublin.csv')
data['Date'] = data['Date'].astype('datetime64[ns]')

In [78]:
for i in tqdm(range(12)):
    for j in range(m[i]):
        tmp = data[data['Date'] == '2010-%02d-%02d' % (i+1, j+1)]['Time']
        
        if 'P' in tmp.values[0]:
            print('2010-%02d-%02d' % (i+1, j+1), tmp.values[0])
            k = 0
            while 'P' in tmp.values[k]:
                data['Date'][tmp.index[k]] = data['Date'][tmp.index[k]] - dateutil.relativedelta.relativedelta(days=1)
                k = k+1

  0%|          | 0/12 [00:00<?, ?it/s]/Users/kaishuai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
 25%|██▌       | 3/12 [00:00<00:00, 13.05it/s]

2010-03-29 11:00 PM
2010-03-30 11:00 PM
2010-03-31 11:00 PM
2010-04-01 11:00 PM
2010-04-02 11:00 PM
2010-04-03 11:00 PM
2010-04-04 11:00 PM
2010-04-05 11:00 PM
2010-04-06 11:00 PM
2010-04-07 11:00 PM
2010-04-08 11:00 PM
2010-04-09 11:00 PM
2010-04-10 11:00 PM
2010-04-11 11:00 PM
2010-04-12 11:00 PM
2010-04-13 11:00 PM
2010-04-14 11:00 PM
2010-04-15 11:00 PM
2010-04-16 11:00 PM
2010-04-17 11:00 PM
2010-04-18 11:00 PM
2010-04-19 11:00 PM
2010-04-20 11:00 PM
2010-04-21 11:00 PM
2010-04-22 11:00 PM
2010-04-23 11:00 PM
2010-04-24 11:00 PM
2010-04-25 11:00 PM
2010-04-26 11:00 PM
2010-04-27 11:00 PM
2010-04-28 11:00 PM


 33%|███▎      | 4/12 [00:01<00:04,  1.91it/s]

2010-04-29 11:00 PM
2010-04-30 11:00 PM
2010-05-01 11:00 PM
2010-05-02 11:00 PM
2010-05-03 11:00 PM
2010-05-04 11:00 PM
2010-05-05 11:00 PM
2010-05-06 11:00 PM
2010-05-07 11:00 PM
2010-05-08 11:00 PM
2010-05-09 11:00 PM
2010-05-10 11:00 PM
2010-05-11 11:00 PM
2010-05-12 11:00 PM
2010-05-13 11:00 PM
2010-05-14 11:00 PM
2010-05-15 11:00 PM
2010-05-16 11:00 PM
2010-05-17 11:00 PM
2010-05-18 11:00 PM
2010-05-19 11:00 PM
2010-05-20 11:00 PM
2010-05-21 11:00 PM
2010-05-22 11:00 PM
2010-05-23 11:00 PM
2010-05-24 11:00 PM
2010-05-25 11:00 PM


 42%|████▏     | 5/12 [00:03<00:05,  1.19it/s]

...

2010-08-27 11:00 PM
2010-08-28 11:00 PM
2010-08-29 11:00 PM
2010-08-30 11:00 PM
2010-08-31 11:00 PM
2010-09-01 11:00 PM
2010-09-02 11:00 PM
2010-09-03 11:00 PM
2010-09-04 11:00 PM
2010-09-05 11:00 PM
2010-09-06 11:00 PM
2010-09-07 11:00 PM
2010-09-08 11:00 PM
2010-09-09 11:00 PM
2010-09-10 11:00 PM
2010-09-11 11:00 PM
2010-09-12 11:00 PM
2010-09-13 11:00 PM
2010-09-14 11:00 PM
2010-09-15 11:00 PM
2010-09-16 11:00 PM
2010-09-17 11:00 PM
2010-09-18 11:00 PM
2010-09-19 11:00 PM
2010-09-20 11:00 PM
2010-09-21 11:00 PM
2010-09-22 11:00 PM
2010-09-23 11:00 PM
2010-09-24 11:00 PM
2010-09-25 11:00 PM
2010-09-26 11:00 PM
2010-09-27 11:00 PM


 75%|███████▌  | 9/12 [00:09<00:04,  1.44s/it]

2010-09-28 11:00 PM
2010-09-29 11:00 PM
2010-09-30 11:00 PM
2010-10-01 11:00 PM
2010-10-02 11:00 PM
2010-10-03 11:00 PM
2010-10-04 11:00 PM
2010-10-05 11:00 PM
2010-10-06 11:00 PM
2010-10-07 11:00 PM
2010-10-08 11:00 PM
2010-10-09 11:00 PM
2010-10-10 11:00 PM
2010-10-11 11:00 PM
2010-10-12 11:00 PM
2010-10-13 11:00 PM
2010-10-14 11:00 PM
2010-10-15 11:00 PM
2010-10-16 11:00 PM
2010-10-17 11:00 PM
2010-10-18 11:00 PM
2010-10-19 11:00 PM
2010-10-20 11:00 PM
2010-10-21 11:00 PM
2010-10-22 11:00 PM
2010-10-23 11:00 PM
2010-10-24 11:00 PM
2010-10-25 11:00 PM
2010-10-26 11:00 PM
2010-10-27 11:00 PM
2010-10-28 11:00 PM


100%|██████████| 12/12 [00:11<00:00,  1.05it/s]

2010-10-29 11:00 PM
2010-10-30 11:00 PM
2010-10-31 11:00 PM


In [79]:
# Datetime
datetime = []
for i in range(len(data)):
    datetime.append(pd.to_datetime(str(data['Date'][i]) + ' ' + data['Time'][i]))
    
data = pd.concat([data, pd.DataFrame(datetime, columns=['DateTime'])], axis=1)

In [80]:
# Transfer to DateTime
data = data[['DateTime', 'T']]
data.to_csv('../data/weather/weather_Dublin_cleaned.csv', index=None)
data = pd.read_csv('../data/weather/weather_Dublin_cleaned.csv')

In [81]:
# Delete rows with T == 0
for i in range(len(data)):
    if data['T'][i] == 0:
        data.drop(index=[i], inplace=True)

In [82]:
data['Miss'] = 0

In [83]:
# Construct the empty DataFrame to store the interpolation
add = pd.DataFrame(columns=['DateTime', 'T', 'Miss'])

for j in range(1, 365*48):
    t = pd.to_datetime('2010-01-01 00:00:00') + dateutil.relativedelta.relativedelta(minutes=30*j)
    if str(t) not in data['DateTime'].values:

        # Use average values as interpolation
        value = (data[data['DateTime'] > str(t)].iloc[0, 1] + data[data['DateTime'] < str(t)].iloc[-1, 1])/2
        tmp = pd.DataFrame([[str(t), value, 1]], columns=['DateTime', 'T', 'Miss'])
        add = pd.concat([add, tmp], axis=0)

In [84]:
data = pd.concat([data, add]).sort_values(by='DateTime')
data.reset_index(inplace=True, drop=True)
data.to_csv('../data/weather/weather_Dublin_cleaned.csv', index=None)

In [85]:
# There are some outliers which make the total number is greater than 17520(48 * 365)
for i in range(12):
    for j in range(m[i]):
        tmp = data[(data['DateTime'] >= '2010-%02d-%02d 00:00:00' % (i+1, j+1)) & (data['DateTime'] < '2010-%02d-%02d 00:00:00' % (i+1, j+2))]
        if len(tmp) != 48:
            print(i+1, j+1, len(tmp))

5 22 49
8 2 49
11 30 49
12 6 49


In [95]:
i=12
j=6

In [96]:
tmp = data[(data['DateTime'] >= '2010-%02d-%02d 00:00:00' % (i, j)) & (data['DateTime'] < '2010-%02d-%02d 00:00:00' % (i, j+1))]

In [97]:
tmp

,DateTime,T,Miss
16275,2010-12-06 00:00:00,27.0,0
16276,2010-12-06 00:30:00,27.0,0
16277,2010-12-06 01:00:00,25.0,0
16278,2010-12-06 01:30:00,25.0,0
16279,2010-12-06 02:00:00,25.0,0
16280,2010-12-06 02:30:00,25.0,0
16281,2010-12-06 03:00:00,23.0,0
16282,2010-12-06 03:30:00,25.0,0
16283,2010-12-06 04:00:00,23.0,0
16284,2010-12-06 04:30:00,23.0,0


In [98]:
data.drop(index=[6779, 10246, 16033, 16293], inplace=True)

In [99]:
len(data)

17520

In [100]:
data.to_csv('../data/weather/weather_Dublin_cleaned.csv', index=None)

## Clean weather data (London)

In [125]:
data = pd.read_csv('../data/weather/weather_London.csv')
data['Date'] = data['Date'].astype('datetime64[ns]')

In [126]:
for i in tqdm(range(12)):
    for j in range(m[i]):
        tmp = data[data['Date'] == '2013-%02d-%02d' % (i+1, j+1)]['Time']
        
        if 'P' in tmp.values[0]:
            print('2013-%02d-%02d' % (i+1, j+1), tmp.values[0])
            k = 0
            while 'P' in tmp.values[k]:
                data['Date'][tmp.index[k]] = data['Date'][tmp.index[k]] - dateutil.relativedelta.relativedelta(days=1)
                k = k+1

  0%|          | 0/12 [00:00<?, ?it/s]/Users/kaishuai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


2013-01-01 11:50 PM
2013-01-02 11:50 PM
2013-01-03 11:50 PM
2013-01-04 11:50 PM
2013-01-05 11:50 PM
2013-01-06 11:50 PM
2013-01-07 11:50 PM
2013-01-08 11:50 PM
2013-01-09 11:50 PM
2013-01-10 11:50 PM
2013-01-11 11:50 PM
2013-01-12 11:50 PM
2013-01-13 11:50 PM
2013-01-14 11:50 PM
2013-01-15 11:50 PM
2013-01-16 11:50 PM
2013-01-17 11:50 PM
2013-01-18 11:50 PM
2013-01-19 11:50 PM
2013-01-20 11:50 PM
2013-01-21 11:50 PM
2013-01-22 11:50 PM
2013-01-23 11:50 PM
2013-01-24 11:50 PM
2013-01-25 11:50 PM
2013-01-26 11:50 PM
2013-01-27 11:50 PM
2013-01-28 11:50 PM
2013-01-29 11:50 PM
2013-01-30 11:50 PM
2013-01-31 11:50 PM


  8%|▊         | 1/12 [00:00<00:10,  1.09it/s]

2013-02-01 11:50 PM
2013-02-02 11:50 PM
2013-02-03 11:50 PM
2013-02-04 11:50 PM
2013-02-05 11:50 PM
2013-02-06 11:50 PM
2013-02-07 11:50 PM
2013-02-08 11:50 PM
2013-02-09 11:50 PM
2013-02-10 11:50 PM
2013-02-11 11:50 PM
2013-02-12 11:50 PM
2013-02-13 11:50 PM
2013-02-14 11:50 PM
2013-02-15 11:50 PM
2013-02-16 11:50 PM
2013-02-17 11:50 PM
2013-02-18 11:50 PM
2013-02-19 11:50 PM
2013-02-21 11:50 PM


 17%|█▋        | 2/12 [00:01<00:08,  1.16it/s]

...

2013-10-27 10:50 PM
2013-10-28 11:50 PM
2013-10-29 11:50 PM
2013-10-30 11:50 PM
2013-10-31 11:50 PM
2013-11-01 11:50 PM
2013-11-02 11:50 PM
2013-11-03 11:50 PM
2013-11-04 11:50 PM
2013-11-05 11:50 PM
2013-11-06 11:50 PM
2013-11-07 11:50 PM
2013-11-09 11:50 PM
2013-11-10 11:50 PM
2013-11-11 11:50 PM
2013-11-12 11:50 PM
2013-11-13 11:50 PM
2013-11-14 11:50 PM
2013-11-15 11:50 PM
2013-11-16 11:50 PM
2013-11-17 11:50 PM
2013-11-18 11:50 PM
2013-11-19 11:50 PM
2013-11-20 11:50 PM
2013-11-21 11:50 PM
2013-11-22 11:50 PM
2013-11-23 11:50 PM
2013-11-24 11:50 PM
2013-11-25 11:50 PM
2013-11-26 11:50 PM
2013-11-27 11:50 PM
2013-11-28 11:50 PM


 92%|█████████▏| 11/12 [00:19<00:01,  1.77s/it]

2013-11-29 11:50 PM
2013-11-30 11:50 PM
2013-12-01 11:50 PM
2013-12-02 11:50 PM
2013-12-03 11:50 PM
2013-12-04 11:50 PM
2013-12-05 11:50 PM
2013-12-06 11:50 PM
2013-12-07 11:50 PM
2013-12-08 11:50 PM
2013-12-09 11:50 PM
2013-12-10 11:50 PM
2013-12-11 11:50 PM
2013-12-12 11:50 PM
2013-12-13 11:50 PM
2013-12-14 11:50 PM
2013-12-15 11:50 PM
2013-12-16 11:50 PM
2013-12-17 11:50 PM
2013-12-18 11:50 PM
2013-12-19 11:50 PM
2013-12-20 11:50 PM
2013-12-21 11:50 PM
2013-12-22 11:50 PM
2013-12-23 11:50 PM
2013-12-24 11:50 PM
2013-12-25 11:50 PM
2013-12-26 11:50 PM
2013-12-27 11:50 PM
2013-12-28 11:50 PM
2013-12-29 11:50 PM
2013-12-30 11:50 PM


100%|██████████| 12/12 [00:20<00:00,  1.48s/it]

2013-12-31 11:50 PM


In [127]:
# Datetime
datetime = []
for i in range(len(data)):
    datetime.append(pd.to_datetime(str(data['Date'][i]) + ' ' + data['Time'][i]))
    
data = pd.concat([data, pd.DataFrame(datetime, columns=['DateTime'])], axis=1)

In [128]:
# Transfer to DateTime
data = data[['DateTime', 'T']]
data.to_csv('../data/weather/weather_London_cleaned.csv', index=None)
data = pd.read_csv('../data/weather/weather_London_cleaned.csv')

In [129]:
# Delete rows with T == 0
for i in range(len(data)):
    if data['T'][i] == 0:
        data.drop(index=[i], inplace=True)

In [130]:
data['Miss'] = 0
data.drop(index=[0], inplace=True)

In [131]:
# Construct the empty DataFrame to store the interpolation
add = pd.DataFrame(columns=['DateTime', 'T', 'Miss'])

for j in range(1, 365*48-1):
    t = pd.to_datetime('2013-01-01 00:20:00') + dateutil.relativedelta.relativedelta(minutes=30*j)
    if str(t) not in data['DateTime'].values:

        # Use average values as interpolation
        value = (data[data['DateTime'] > str(t)].iloc[0, 1] + data[data['DateTime'] < str(t)].iloc[-1, 1])/2
        tmp = pd.DataFrame([[str(t), value, 1]], columns=['DateTime', 'T', 'Miss'])
        add = pd.concat([add, tmp], axis=0)

In [132]:
data = pd.concat([data, add]).sort_values(by='DateTime')
data.reset_index(inplace=True, drop=True)

In [134]:
# Add the last point
tmp = pd.DataFrame([['2013-12-31 23:50:00', 48, 0]], columns=['DateTime', 'T', 'Miss'])
data = pd.concat([data, tmp], axis=0)

In [136]:
data.to_csv('../data/weather/weather_London_cleaned.csv', index=None)

## Combine Cork and Dublin

In [51]:
df1 = pd.read_csv('../data/weather/weather_Cork_cleaned.csv')
df2 = pd.read_csv('../data/weather/weather_Dublin_cleaned.csv')

In [53]:
# Detect continuous missing values
def detect_c_miss(df):
    c_miss = []
    for i in range(len(df)):
        if df['Miss'][i] == 0:
            c_miss.append(0)
        else:
            count = 1
            k = i
            while k-1>=0 and df['Miss'][k-1] == 1:
                count = count + 1
                k = k - 1

            k = i
            while k+1<len(df) and df['Miss'][k+1] == 1:
                count = count + 1
                k = k + 1

            if count > 2:
                c_miss.append(1)
            else:
                c_miss.append(0)
    return c_miss

In [54]:
df1 = pd.concat([df1, pd.DataFrame(detect_c_miss(df1), columns=['Continous Miss'])], axis=1)
df2 = pd.concat([df2, pd.DataFrame(detect_c_miss(df2), columns=['Continous Miss'])], axis=1)

In [55]:
data = pd.DataFrame(columns=['DateTime', 'T'])

for i in tqdm(range(len(df1))):
    if df1['Continous Miss'][i] == 0 and df2['Continous Miss'][i] == 0:
        value = (df1['T'][i] + df2['T'][i]) / 2
    elif df1['Continous Miss'][i] == 0 and df2['Continous Miss'][i] == 1:
        value = df1['T'][i]
    elif df1['Continous Miss'][i] == 1 and df2['Continous Miss'][i] == 0:
        value = df2['T'][i]
    else:
        value = (df1['T'][i] + df2['T'][i]) / 2
    tmp = pd.DataFrame([[df1['DateTime'][i], value]], columns=['DateTime', 'T'])
    data = pd.concat([data, tmp], axis=0)

100%|██████████| 17520/17520 [00:18<00:00, 929.98it/s]


In [57]:
data.to_csv('../data/weather/weather_Ireland_cleaned.csv', index=None)